# Advent of code 2023

## [Day 4](https://adventofcode.com/2023/day/4): Scratchcards

### Solution in python

#### part 1

In [ ]:
file = open('input.txt', 'r')
part1 = 0
for ix, line in enumerate(file):
    card, line = line.split(":")
    card = int(card.split(" ")[-1])
    winning, had =  [[int(num.strip()) for num in l.split(" ") if num] for l in line.split("|")]
    winning_had = [x for x in had if x in winning]
    score = 0 if not winning_had else 2**(len(winning_had)-1)
    part1 += score
part1

#### part 2

In [ ]:
file = open('input.txt', 'r')
scratch_cards = {}
for ix, line in enumerate(file):
    card, line = line.split(":")
    card = int(card.split(" ")[-1])
    winning, had =  [[int(num.strip()) for num in l.split(" ") if num] for l in line.split("|")]
    winning_had = [x for x in had if x in winning]
    matches = len(winning_had)
    scratch_cards[card] = {"matches": matches, "num": 1}

In [ ]:
for card_num in scratch_cards:
    for offset in range(1, scratch_cards[card_num]['matches'] + 1):
        scratch_cards[card_num + offset]['num'] += scratch_cards[card_num]['num']

part2 = 0
for card_num in scratch_cards:
    part2 += scratch_cards[card_num]['num']
part2

### Solution with Neo4j

In [60]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

2.5.5


#### preprocessing data

In [127]:
file = open('input.txt', 'r')
scratch_cards = [] 
for ix, line in enumerate(file):
    card, line = line.split(":")
    card = int(card.split(" ")[-1])
    winning, had =  [[int(num.strip()) for num in l.split(" ") if num] for l in line.split("|")]
    scratch_cards.append({"card":card, "winning":winning, "had":had})

#### part 1

In [128]:
query ="""
UNWIND $scratch_cards AS card
WITH size([h IN card.had WHERE h IN card.winning]) AS wins
WITH CASE wins WHEN 0 THEN 0 ELSE toInteger(2^(wins-1)) END AS score
RETURN sum(score) AS part1
"""
gds.run_cypher(query, {"scratch_cards":scratch_cards})

,part1
0,24175


#### part 2

In [130]:
query ="""
UNWIND $scratch_cards AS card
WITH size([h IN card.had WHERE h IN card.winning]) AS wins
WITH collect(wins) AS wins
WITH [ix IN range(0, size(wins)-1) | [1,wins[ix]]] AS accumulator
WITH reduce(acc=accumulator, ix IN range(0, size(accumulator)-1) |
    [ix_acc IN range(0,size(acc)-1) | CASE ix < ix_acc <= ix + acc[ix][1]
        WHEN TRUE THEN [ acc[ix_acc][0] + acc[ix][0]  , acc[ix_acc][1]]
        ELSE acc[ix_acc] END ]) AS processed
UNWIND processed AS card
RETURN sum(card[0]) AS part2
"""
gds.run_cypher(query, {"scratch_cards":scratch_cards})

,part2
0,18846301
